## Chi-square test of independence

In [2]:
import sys
sys.path.append('/data/joramvandriel/ai-monitoring-streamlit-dashboard/')
from drift_dataset import DriftDataset
from pathlib import Path
import numpy as np
import pandas as pd
from IPython import embed

In [24]:
from scipy.stats import chi2_contingency 

In [3]:
binnenvaart_pad = Path.home()/'share'/'Binnenvaart'/'Monitoring'

In [4]:
def mark_datasets(df: pd.DataFrame):
    """Post processing function to add markings for labelled and unlabelled samples in each dataset"""
    
    def mark_labels(df: pd.DataFrame):
        """To be applied over groups, marking the datasets"""
        df = df.copy()
        [idx] = df.dataset_index.unique()
        if idx == 0:
            train_label = 'train'
            oos_label = 'oos'
        else:
            train_label = f'labelled_{idx}'
            oos_label = f'unlabelled_{idx}'

        begindatum = max(df.Begindatum)
        mask = df.Begindatum < begindatum
        df['dataset_markering'] = None
        df.loc[mask, 'dataset_markering'] = train_label
        df.loc[~mask, 'dataset_markering'] = oos_label
        
        return df
    
    df = df.copy()
    df = df.groupby(['dataset_index']).apply(mark_labels)
    df = df.reset_index(drop=True)
    
    return df

In [5]:
print('Post processing data...')
# features uit top 10 Gini halen
gini = pd.read_csv(binnenvaart_pad/'meta'/'meanDecreaseGini.csv')
features = gini.Variable[:10].tolist() + ['Begindatum', 'dataset_markering'] # laatste wordt toegevoegd door mark_datasets()
dd = DriftDataset(binnenvaart_pad/'data', post_process=mark_datasets)
dd.keep_columns(features) 

Post processing data...


In [9]:
dd.numeric_columns()

['Lat_insploc',
 'Diepgang_vaartuig',
 'Lengte_vaartuig',
 'Breedte_vaartuig',
 'Bouwjaar',
 'laatstAlgemeen',
 'Lon_insploc',
 'Tonnage']

`drift_dataset` functie om categorische variabelen te pakken.

In [55]:
dd.categorical_columns(exclude=["dataset_markering"])

['soortBeroepsvaartuig', 'Begindatum', 'insp_weekdag']

In [7]:
data = dd.dataset.copy()
data.shape

(674438, 14)

In [18]:
# we pakken even twee tijdspunten
tt = ['train','labelled_10']
df = data[data.dataset_markering.isin(tt)]

Voor de chi-square hebben we een frequency of 'contingency' table nodig. Dus groeperen op `dataset_markering` en één categorische feature, en dan een pivot wide doen.

In [20]:
df.groupby(['dataset_markering','soortBeroepsvaartuig']).size().reset_index(name='count').pivot(index="dataset_markering",columns="soortBeroepsvaartuig",values="count")

soortBeroepsvaartuig,Bunkerboot,Duwboot,Motortankschip,Motorvrachtschip,Overige vaartuigen,Passagiersschip,Patrouilleboot,Sleepboot,Tankbak,Veerpont,Vissersboot,Vrachtbak,Werkschip,onbekend
dataset_markering,,,,,,,,,,,,,,
labelled_10,130.0,164.0,1191.0,2116.0,52.0,291.0,2.0,155.0,37.0,373.0,2.0,314.0,29.0,27.0
train,133.0,202.0,1299.0,2760.0,69.0,469.0,NaN,214.0,40.0,566.0,5.0,603.0,40.0,174.0


`pandas` blijkt daar zelf een shortcut functie voor te hebben:

In [26]:
pd.crosstab(df.dataset_markering,df.soortBeroepsvaartuig)

soortBeroepsvaartuig,Bunkerboot,Duwboot,Motortankschip,Motorvrachtschip,Overige vaartuigen,Passagiersschip,Patrouilleboot,Sleepboot,Tankbak,Veerpont,Vissersboot,Vrachtbak,Werkschip,onbekend
dataset_markering,,,,,,,,,,,,,,
labelled_10,130,164,1191,2116,52,291,2,155,37,373,2,314,29,27
train,133,202,1299,2760,69,469,0,214,40,566,5,603,40,174


In [27]:
print(chi2_contingency(pd.crosstab(df.dataset_markering,df.soortBeroepsvaartuig).to_numpy())[0:3])

(144.20282496315744, 3.021096068697427e-24, 13)


Een hele lage p-value; dus de alternatieve hypothese is waar; maar die is hier juist dat de variabelen afhankelijk zijn, oftewel een correlatie hebben. Er wordt dus niet voor een verschil getest maar juist een overeenkomst.

Nu op alle features in `drift_detector` format.

Daarvoor eerst kijken of het werkt met een `x[fi], y[fi]` notatie.

In [28]:
refset = data.loc[data['dataset_markering']=='train']
refset

,soortBeroepsvaartuig,dataset_index,dataset_date,Lat_insploc,Diepgang_vaartuig,Lengte_vaartuig,Breedte_vaartuig,insp_weekdag,Bouwjaar,Begindatum,laatstAlgemeen,dataset_markering,Lon_insploc,Tonnage
0,onbekend,0,2020-03-16,52.191315,2.0,49.0,7.0,Fri,1965,2015-05-22,3680,train,4.528120,1037.0
1,onbekend,0,2020-03-16,52.175522,2.0,49.0,7.0,Fri,1965,2015-05-22,3680,train,4.517810,1037.0
2,onbekend,0,2020-03-16,51.636654,2.0,49.0,7.0,Wed,1965,2017-09-27,3680,train,4.247420,1037.0
3,onbekend,0,2020-03-16,0.000000,2.0,49.0,7.0,Thu,1965,2018-01-11,3680,train,0.000000,1037.0
4,onbekend,0,2020-03-16,51.707363,2.0,49.0,7.0,Wed,1965,2016-09-07,3680,train,4.589214,1037.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37858,onbekend,0,2020-03-16,0.000000,2.0,49.0,7.0,Tue,1965,2015-07-28,3680,train,0.000000,1037.0
37860,onbekend,0,2020-03-16,52.195004,2.0,49.0,7.0,Fri,1965,2015-08-07,3680,train,6.538424,1037.0
37875,onbekend,0,2020-03-16,0.000000,2.0,49.0,7.0,Thu,1965,2015-05-28,3680,train,0.000000,1037.0
37876,onbekend,0,2020-03-16,52.060761,2.0,11.0,7.0,Tue,2013,2015-04-21,3680,train,4.661255,1037.0


In [38]:
features = list(data.select_dtypes(exclude='number').columns)

In [41]:
features.remove('dataset_markering')
features.remove('dataset_date')

In [50]:
def chisq(x,y):
    features = list(x.select_dtypes(exclude='number').columns)
    features.remove('dataset_markering')
    features.remove('dataset_date')
    df = pd.concat([x,y],axis=0)
    result = { fi : chi2_contingency(pd.crosstab(df.dataset_markering,df[fi]).to_numpy())[0] for fi in features }
    return pd.Series(result)   

In [48]:
x=data.loc[data['dataset_markering']=='labelled_10'].copy()
y=refset.copy()
df = pd.concat([x,y],axis=0)

In [51]:
chi = data.groupby(['dataset_index', 'dataset_date']).apply(chisq, y=refset)

In [52]:
chi

,,soortBeroepsvaartuig,insp_weekdag,Begindatum
dataset_index,dataset_date,,,
0,2020-03-16,5212.643760,33479.988954,44455.000000
1,2021-02-01,5241.987567,34201.583418,50416.918203
2,2021-09-28,5449.195781,29968.705346,55860.071502
3,2021-10-06,5455.350382,31106.844068,55948.432165
4,2021-10-28,5471.532187,34127.085543,56393.757945
5,2021-11-10,5523.027570,30991.791402,56547.224734
6,2021-11-10,5523.027570,30991.791402,56547.224734
7,2021-11-15,5523.881616,32037.023028,56629.955420
8,2021-12-01,3234.025742,28296.001063,37544.000000


In [53]:
columns = data.select_dtypes(exclude='number').columns

In [54]:
columns

Index(['soortBeroepsvaartuig', 'dataset_date', 'insp_weekdag', 'Begindatum',
       'dataset_markering'],
      dtype='object')

In [ ]:
if exclude is not None:
    columns = set(columns).difference(set(exclude))  # difference takes what is in the first and not in the second set
# adding or removing index columns (self.keep)
if incl_indices:
    columns = set(columns).union(self.keep)
else:
    columns = set(columns).difference(self.keep)